### Weather, Traffic, adjusted time, and categorical values (Variable selection is applied to this data before model 3)

#import needed libraries & float format definition
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy import stats
from scipy.stats import norm
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
import time
import datetime

### Table_2, This table is created after running models table one and doing some more EDA, some more work is done in terms of dummy variable, feature creation, feature selection 

In [184]:
Final_Table_2 = pd.read_csv('Merged_Table_1.csv')
Final_Table_2 = Final_Table_2.drop(columns= 'Unnamed: 0')
Final_Table_2.head()

,index,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,Trip_count_PerDay,Trip_count_Permonth,Trip_count_PerYear,origin_city_condn,dest_city_condn
0,0,2018-01-01,WN,5705,"Tampa, FL","Albany, NY",1530,28.0,12.0,1610.0,...,0,160,153.0,134.0,1130,1.001351,30.875000,61.750000,low_rainny,sunny
1,1,2018-01-01,WN,988,"New Orleans, LA","Houston, TX",625,-2.0,9.0,632.0,...,0,75,69.0,57.0,302,17.332432,534.416667,1068.833333,cloundy,sunny
2,2,2018-01-01,WN,1236,"New Orleans, LA","Orlando, FL",515,-1.0,8.0,522.0,...,0,100,86.0,70.0,551,5.255405,162.041667,324.083333,cloundy,low_rainny
3,3,2018-01-01,WN,233,"New Orleans, LA","Oakland, CA",1530,16.0,8.0,1554.0,...,0,290,283.0,271.0,1903,0.959459,29.583333,59.166667,cloundy,sunny
4,4,2018-01-01,WN,446,"New Orleans, LA","St. Louis, MO",1805,3.0,6.0,1814.0,...,0,105,125.0,83.0,604,1.981081,61.083333,122.166667,cloundy,sunny


In [185]:
#Change Wheels off and on into difference and drop them 
Final_Table_2['Minutes_to_wheels_off'] = Final_Table_2['wheels_off'] - Final_Table_2 ['crs_dep_time']
Final_Table_2['Minutes_from_wheels_on'] = Final_Table_2['wheels_on'] - Final_Table_2 ['crs_arr_time']


In [186]:
# Drop Flight number  mkt_carrier_fl_num
Final_Table_2 = Final_Table_2.drop(columns = 'wheels_off')
Final_Table_2 = Final_Table_2.drop(columns = 'wheels_on')

In [187]:
# Changing Fl_Date to engineer split Datetime Features 
Final_Table_2['year'] = pd.DatetimeIndex(Final_Table_2['fl_date']).year
Final_Table_2['month'] = pd.DatetimeIndex(Final_Table_2['fl_date']).month
Final_Table_2['day'] = pd.DatetimeIndex(Final_Table_2['fl_date']).day
Final_Table_2['day_Week'] = pd.DatetimeIndex(Final_Table_2['fl_date']).dayofweek
# Dropping Fl_Date and Year
Final_Table_2 = Final_Table_2.drop(columns= ['fl_date', 'year','month'])

In [188]:
# lab encoder --> change from categorical to number
# origin_city_condn
# dest_city_condn
# origin_city_name
# dest_city_name
# mkt_unique_carrier

# Import label encoder
from sklearn.preprocessing import LabelEncoder
categorical_features = ['origin_city_condn', 'dest_city_condn', 'origin_city_name','dest_city_name','mkt_unique_carrier']
le = LabelEncoder()

# Convert the variables to numerical
for i in range(len(categorical_features)):
    new = le.fit_transform(Final_Table_2[categorical_features[i]])
    Final_Table_2[categorical_features[i]] = new

In [189]:
# Trip_count_PerDay only drop month and year 
Final_Table_2 = Final_Table_2.drop(columns = ['Trip_count_Permonth','Trip_count_PerYear'])
Final_Table_2['Trip_count_PerDay'] = Final_Table_2['Trip_count_PerDay'].round()

In [190]:
Final_Table_2

,index,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,...,actual_elapsed_time,air_time,distance,Trip_count_PerDay,origin_city_condn,dest_city_condn,Minutes_to_wheels_off,Minutes_from_wheels_on,day,day_Week
0,0,9,5705,269,4,1530,28.0,12.0,7.0,1810,...,153.0,134.0,1130,1.0,2,6,80.0,14.0,1,0
1,1,9,988,194,125,625,-2.0,9.0,3.0,740,...,69.0,57.0,302,17.0,0,6,7.0,-11.0,1,0
2,2,9,1236,194,210,515,-1.0,8.0,8.0,755,...,86.0,70.0,551,5.0,0,2,7.0,-23.0,1,0
3,3,9,233,194,205,1530,16.0,8.0,4.0,1820,...,283.0,271.0,1903,1.0,0,6,24.0,5.0,1,0
4,4,9,446,194,274,1805,3.0,6.0,36.0,1950,...,125.0,83.0,604,2.0,0,6,9.0,-13.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,14595,9,5451,195,139,1835,94.0,7.0,3.0,2100,...,165.0,155.0,1107,6.0,0,6,181.0,51.0,31,1
14596,14596,9,2038,195,52,1155,-4.0,13.0,44.0,1335,...,165.0,108.0,725,54.0,0,3,49.0,-83.0,31,1
14597,14597,9,3255,195,197,1235,0.0,16.0,3.0,1510,...,198.0,179.0,1183,8.0,0,6,16.0,-60.0,31,1
14598,14598,9,4880,195,281,845,10.0,12.0,4.0,1150,...,174.0,158.0,1010,13.0,0,6,62.0,-5.0,31,1


In [192]:
Final_Table_2.to_csv('Final_Table_2.csv')